In [26]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
from datetime import timedelta

# Read the CSV file
df = pd.read_csv('sorted_result.csv')

# Convert start_time and end_time to datetime
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

# Sort the dataframe by start_time
df = df.sort_values(by=["start_time"], ascending=True)

# Get the min and max values of mean_snr_db for consistent color scale
cmin = df['mean_snr_db'].min()
cmax = df['mean_snr_db'].max()

# Create the Dash app
app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='map-graph'),
    html.Div([
        html.Label('Select Station:'),
        dcc.Dropdown(
            id='station-dropdown',
            options=[{'label': i, 'value': i} for i in df['station'].unique()] + [{'label': 'All', 'value': 'All'}],
            value=['All'],
            multi=True
        ),
        html.Label('Select Cluster:'),
        dcc.Dropdown(
            id='cluster-dropdown',
            options=[{'label': i, 'value': i} for i in df['cluster'].unique()] + [{'label': 'All', 'value': 'All'}],
            value=['All'],
            multi=True
        )
    ], style={'width': '100%', 'display': 'inline-block'})
])

@app.callback(
    Output('map-graph', 'figure'),
    [Input('station-dropdown', 'value'),
     Input('cluster-dropdown', 'value')]
)
def update_graph(selected_stations, selected_clusters):
    filtered_df = df.copy()
    
    if not selected_stations or 'All' in selected_stations:
        selected_stations = df['station'].unique()
    if not selected_clusters or 'All' in selected_clusters:
        selected_clusters = df['cluster'].unique()
    
    filtered_df = filtered_df[
        filtered_df['station'].isin(selected_stations) &
        filtered_df['cluster'].isin(selected_clusters) 
    ]

    # Check if the filtered dataframe is empty
    if filtered_df.empty:
        # Return an empty figure with a message
        return go.Figure().add_annotation(
            x=0.5, y=0.5,
            text="No data available for the selected criteria",
            font=dict(size=20),
            showarrow=False,
            xref="paper", yref="paper"
        )

    # Prepare the animation frames
    frames = []
    for current_time in filtered_df['start_time'].sort_values().unique():
        # Filter data for events that have started but not ended
        frame_data = filtered_df[
            (filtered_df['start_time'] <= current_time) & 
            (filtered_df['end_time'] > current_time)
        ]
        
        frame = go.Frame(
            data=[go.Scattergeo(
                lat=frame_data['latitude'],
                lon=frame_data['longitude'],
                text=frame_data['station'],
                mode='markers',
                marker=dict(
                    size=10,
                    opacity=0.7,
                    color=frame_data['mean_snr_db'],
                    colorscale=px.colors.sequential.Viridis,
                    cmin=cmin,
                    cmax=cmax,
                    colorbar=dict(
                        title='Mean SNR (dB)',
                        thickness=15,
                        len=0.9,
                        x=1.02,
                        xanchor='left',
                        outlinewidth=0
                    )
                ),
                hovertemplate=(
                    "Station: <b>%{text}</b><br><br>" +
                    "Cluster: %{customdata[0]}<br>" +
                    "Mean Peak(dB): %{customdata[1]:.2f}<br>" +
                    "Max Peak(dB): %{customdata[2]:.2f}<br>" +
                    "Mean Duration(s): %{customdata[3]:.2f}<br>" +
                    "Mean Background(dB): %{customdata[4]:.2f}<br>" +
                    "Max Background(dB): %{customdata[5]:.2f}<br>" +
                    "Mean SNR(dB): %{customdata[6]:.2f}<br>" +
                    "Max SNR(dB): %{customdata[7]:.2f}<br>" +
                    "Max Duration(s): %{customdata[8]:.2f}<br>" +
                    "Start Time: %{customdata[9]}<br>" +
                    "End Time: %{customdata[10]}<br><extra></extra>"
                ),
                customdata= frame_data[
                    [
                        'cluster', 'mean_peak_db', 'max_peak_db', 'mean_duration_sec', 
                        'mean_background_db', 'max_background_db', 'mean_snr_db', 
                        'max_snr_db', 'max_duration_sec', 'start_time', 'end_time'
                    ]
                ]
            )],
            name=str(current_time)
        )
        frames.append(frame)

    # Create the base map
    fig = go.Figure(
        data=[frames[0].data[0]] if frames else [],
        frames=frames,
        layout=go.Layout(
            title='Infrasound Events in North America',
            geo=dict(
                scope='north america',
                projection_type='natural earth',
                showland=True,
                landcolor='rgb(243, 243, 243)',
                countrycolor='rgb(204, 204, 204)',
                lataxis_range=[20, 80],
                lonaxis_range=[-170, -50],
                showcountries=True,
                showsubunits=True,
                subunitcolor="Blue",
                showlakes=True,
                lakecolor="LightBlue",
                showrivers=True,
                rivercolor="LightBlue"
                

            ),
            updatemenus=[dict(
                type='buttons',
                showactive=False,
                buttons=[dict(
                    label='Play',
                    method='animate',
                    args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True)]
                )]
            )],
            coloraxis=dict(colorscale=px.colors.sequential.Viridis, cmin=cmin, cmax=cmax),
            width=1300,
            height=800,
            margin=dict(r=80, l=80, t=80, b=80)
        )
    )

    # Add slider only if there are frames
    if frames:
        fig.update_layout(
            sliders=[dict(
                active=0,
                steps=[dict(
                    method='animate',
                    args=[[f.name], dict(mode='immediate', frame=dict(duration=500, redraw=True), transition=dict(duration=300))],
                    label=f.name
                ) for f in frames],
                transition=dict(duration=300),
                x=0,
                y=0,
                currentvalue=dict(font=dict(size=12), prefix='Time: ', visible=True, xanchor='center'),
                len=1.0
            )]
        )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=3322)